Kütüphaneleri Ekle

In [ ]:
import os                               # operating system
import tensorflow as tf                 # tensorFlow
import cv2                              # openCV-Computer Vision
import numpy as np                      # numpyArray
from matplotlib import pyplot as plt    # plotting library
import time                             # time stamps
import mediapipe as mp                  # mediapipe

mp_holistic = mp.solutions.holistic # Holistic create
mp_drawing = mp.solutions.drawing_utils 

# Functions
# Function will extract frames from a video using OpenCV to intialise mediapipe features
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # image is no longer writeable
    results = model.process(image)                 # make prediction
    image.flags.writeable = True                   # image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    STATIC_IMAGE_MODE = True                       # detection runs every input image
    ENABLE_SEGMENTATION = True                     # generates the segmentation mask
    return image, results

# Function will draw landamrks for face, pose, left and right hands on frames to process
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)   # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)  # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    
# customising the drawn landmarks
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(255,255,255), thickness=1, circle_radius=1)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(255,255,0), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=1)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=1)
                             ) 

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

TEST

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Start MediaPipe modules
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic


cap = cv2.VideoCapture(0)
ret, frame = cap.read()
window_width, window_height = 480, 360
"""
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Renk formatını BGR'dan RGB'ye dönüştür
    image.flags.writeable = False
    results = model.process(image)                 # Tespit yap
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Renk formatını RGB'den BGR'ye geri dönüştür
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
"""

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while ret:
        # Görüntüyü MediaPipe formatına dönüştür
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = holistic.process(image)
        image.flags.writeable = True
        
       
        frame_height, frame_width, _ = frame.shape
    
        if results.face_landmarks:
            # Calculate face landmarks
            landmarks = results.face_landmarks.landmark
            x_coords = [lm.x for lm in landmarks]
            y_coords = [lm.y for lm in landmarks]
            x_center = int(np.mean(x_coords) * frame_width)
            y_center = int(np.mean(y_coords) * frame_height)

            # Adjust window
            start_x = max(x_center - window_width // 2, 0)
            end_x = start_x + window_width
            if end_x > frame_width:
                end_x = frame_width
                start_x = frame_width - window_width

            start_y = max(y_center - window_height // 2, 0)
            end_y = start_y + window_height
            if end_y > frame_height:
                end_y = frame_height
                start_y = frame_height - window_height

            # Calculate the coordinates of the center of the face in the new window
            new_x_center = x_center - start_x
            new_y_center = y_center - start_y

            window = frame[start_y:end_y, start_x:end_x]
            window=cv2.flip(window,1)

            image_y, results = mediapipe_detection(window, holistic)
            
            # Draw the landmarks
            draw_styled_landmarks(image_y, results)
            # Display the coordinate information in a new window
            coordinates_text = f"X: {new_x_center}, Y: {new_y_center}"
            cv2.putText(window, coordinates_text, (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.imshow('Tüm Vücut Tespiti', image_y)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        ret, frame = cap.read()
cap.release()
cv2.destroyAllWindows()


Generate images consisting of 30 frames from the keypoints and save them to the specified folder (TEST)

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
os.mkdir("OrnekFrame1")
# Start MediaPipe Holistic model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)
window_width, window_height = 480, 360
"""
# MediaPipe modelini kullanarak tespit yapma fonksiyonu
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

# Function to draw landmarks
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
"""
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for frame_num in range(30):
        ret, frame = cap.read()
        if not ret:
            break
        # # Perform detection"
        # Convert the image to MediaPipe format
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = holistic.process(image)
        image.flags.writeable = True
        image=cv2.flip(image,1)
       
        frame_height, frame_width, _ = frame.shape
        
        # If a face is detected, set up and display the window
        if results.face_landmarks:
            # Calculate the average of the face landmarks
            landmarks = results.face_landmarks.landmark
            x_coords = [lm.x for lm in landmarks]
            y_coords = [lm.y for lm in landmarks]
            x_center = int(np.mean(x_coords) * frame_width)
            y_center = int(np.mean(y_coords) * frame_height)

            # Set up the window
            start_x = max(x_center - window_width // 2, 0)
            end_x = start_x + window_width
            if end_x > frame_width:
                end_x = frame_width
                start_x = frame_width - window_width

            start_y = max(y_center - window_height // 2, 0)
            end_y = start_y + window_height
            if end_y > frame_height:
                end_y = frame_height
                start_y = frame_height - window_height

            # Calculate the coordinates of the center of the face in the new window
            new_x_center = x_center - start_x
            new_y_center = y_center - start_y

            
            # Create and display the new window
            window = frame[start_y:end_y, start_x:end_x]
            window=cv2.flip(window,1)

             # Perform detection
            image_y, results = mediapipe_detection(window, holistic)
        
         # Create an empty background
        blank_image = np.zeros(image_y.shape, dtype=np.uint8)
        # Draw the landmarks on an empty background.
        draw_styled_landmarks(blank_image, results)
        # Show the results
        cv2.imshow('Body Detection with the Holistic Model', blank_image)
        # save the frame
        cv2.imwrite(f'OrnekFrame/frame_{frame_num}.png', blank_image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Selection of Words and Setting Up Folders

In [ ]:
# ave Folder
DATA_PATH = os.path.join('DATASET1') 

# Kelime Listesi
actions=np.array(['sinifta']) 
# 'gulegule' 'gunaydin' 'hosbulduk' 'hosgeldiniz' 'merhaba' 5
# 'ozurdilerim' 'neden' 'tesekkurler' 'yapmak' 9
# 'siyah' 'kopek' 'neseli' 'beyaz' 'degil' 'hangi' 'adam' 'heyecanli' 'tembel' 18
# 'saygi' 'var' 'efendi' 'dede' 'tanimiyor' 'yer' 'kiz' kedi' 'bir' 'sinifta' 28
# 'bes' 'ogrenci' 'akilli' 'sinirli' 'yaramaz' 'hediye' 'kutuda'
# 'burda' 'yapiyor' 'dondurmaci' 'engelli'
# 'sinif' 'sakin' 'kim' 'gordu' 'ne' 'yardim' 'paylaşmak'

no_sequences = 10     # Number of videos per word.
sequence_length = 30  # Number of frames per video

# ---------------------------------------------
# DATA_PATH = os.path.join('DATASET1') 
# 'Siyah' 'Kopek' 'Neseli' 'Beyaz' 'Degil' 'Hangi' 'Adam' 'Heyecanli' 'Tembel' mehmet emin
# 'Saygi' 'Var' 'Efendi' 'Dede' 'Tanimiyor' 'Yer' *'Verdi'* 'Kiz' *'Getirdi'*
# 'Kedi' *'Ne'* *'Nasil'* 'Bir' 'Sinifta' 'Bes' 'Ogrenci' *'Merakli' 'Akilli'
# 'Sinirli' 'Yaramaz' 'Hediye' 'Kutuda' 'Burda' 'Yapiyor' 'Dondurmaci' 'Engelli'
# 'Sinif' 'Sakin'
# 'OzurDilerim' 'Neden' +2 'Tesekkurler' 'Yapmak' 'Kim' 
# 'Gulegule','Gunaydin','Hosbulduk','Hosgeldiniz','Merhaba'

In [ ]:
# Create the folders.
for action in actions: 
    for sequence in range(no_sequences):    #emir kaan (no_sequences), berkant (10,20), mehmet emin (20,30), eren (30,40), kayra (40,50)
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

Start Creating the Dataset

In [ ]:
cap = cv2.VideoCapture(0)
window_width, window_height = 480, 360
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Loop for the number of example words.
    for action in actions:
        # Loop for the number of videos
        for sequence in range(no_sequences):  #emir kaan (no_sequences), berkant (10,20), mehmet emin (20,30), eren (30,40), kayra (40,50)
            # Loop for the number of video frames
            for frame_num in range(sequence_length):                
                # Kamera oku
                ret, frame = cap.read()
                
                # Convert the image to MediaPipe format
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = holistic.process(image)
                image.flags.writeable = True


                frame_height, frame_width, _ = frame.shape

                # If a face is detected, set up and display the window.
                if results.face_landmarks:
                    # “Calculate the average of the face landmarks
                    landmarks = results.face_landmarks.landmark
                    x_coords = [lm.x for lm in landmarks]
                    y_coords = [lm.y for lm in landmarks]
                    x_center = int(np.mean(x_coords) * frame_width)
                    y_center = int(np.mean(y_coords) * frame_height)

                    start_x = max(x_center - window_width // 2, 0)
                    end_x = start_x + window_width
                    if end_x > frame_width:
                        end_x = frame_width
                        start_x = frame_width - window_width

                    start_y = max(y_center - window_height // 2, 0)
                    end_y = start_y + window_height
                    if end_y > frame_height:
                        end_y = frame_height
                        start_y = frame_height - window_height

                    # Calculate the coordinates of the center of the face in the new window
                    new_x_center = x_center - start_x
                    new_y_center = y_center - start_y


                    # Create and display the new window.
                    window = frame[start_y:end_y, start_x:end_x]
                    window=cv2.flip(window,1)
                     # Tespit yap
                    image, results = mediapipe_detection(window, holistic)

                    # Draw the landmarks.
                    draw_styled_landmarks(image, results)
                    

                # For transitioning to the new word
                if frame_num == 0: 
                    cv2.putText(image, 'CAPTURING STARTS', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, ' {} Kelimesi icin Video Sayisi {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)                   
                    cv2.imshow('OpenCV Beslemesi', image)
                    # 2 saniye bekle
                    cv2.waitKey(1000)
                else: 
                    cv2.putText(image, '{} Kelimesi icin Video Sayisi {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)                
                    cv2.imshow('OpenCV Beslemesi', image)
                
                # Detect, flatten, and merge the keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                # Save the flattened keypoint data for each frame. 
                np.save(npy_path, keypoints)
                
                blank_image = np.zeros(image.shape, dtype=np.uint8)
                draw_styled_landmarks(blank_image, results)
                cv2.imwrite(f'{npy_path}.png', blank_image)

               # Çıkış
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()